In [1]:
# 한국어 영어 번역 데이터셋
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata

# 실제 프로젝트 AI hub 기타 등등..

# 1.
korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "저는 학생입니다",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "우리는 친구예요"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "i am a student",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "we are friends"
]

In [2]:
# 텍스트 전처리
# unicode 정규화, 특수문자처리
# NFD  Normalization Form Decomposition -> 분해 가능한 모든 문자를 분해한다
import unicodedata
unicodedata.normalize('NFD',"e'")

"e'"

###### 같은 문장처럼 보이지만 내부적으로 다른 바이트 조합을 갖는 경우 -> 동일한 내부표현으로 바꿔주는 과정

- 유티코드
    - 전 세계의 모든 문자를 하나의 표준 코드로 표현하기 위한 체계

- 유티코드 정규화를 해야되는 이유
    - 문자 하나가 여러 방식으로 표현될 수 있기 때문이다.
    - 그 여러문자들을 하나의 기준으로 통합하고자 사용되고 있다.
###### ucicode_normalizer_md 참고

In [ ]:
# 1.1
def preprocess_sentence(sentence, is_korean=False):
    '''
    Unicode 정규화, 특수문자처리
    Args :
        sentence  : 원본
        is_korean : 한국어 여부
    return :
        전처리된 문장
    '''
    
    sentence = unicodedata.normalize('NFD',sentence)
    if not is_korean:
        sentence = sentence.lower()
    
    # 정규식으로 특수문자 전후에 공백 추가
    # 안녕하세요! -> 안녕하세요 !
    # -> 공백을 추가하는 이유 ^
    # -> 토큰화를 쉽게 하기 위해, 공백을 기준으로 단어분리
    
    # r("[? . ! ,]")
    # 문자중에 ? . ! ,가  나오면 해당문자를 그룹으로 캡처한다. ^
    
    # r" \1 " 캡처한 문자 ( \1 ) 앞뒤로 공백을 하나씩 넣는다.
    sentence = re.sub(r"([? . ! ,])",r" \1 ",sentence)
    sentence = re.sub(r'[" "]'," ",sentence) # 이중 공백 처리
    
    # 시작 종료 토큰 추가
    sentence = '<start>' + sentence + '<end>'
    return sentence

In [4]:
korean_processed = [preprocess_sentence(kor) for kor in korean_sentences]
english_processed = [preprocess_sentence(eng) for eng in english_sentences]

In [ ]:
# 문장을 정수 시퀀스로 변환

# 1.2 토크나이저
def create_tokenizer(sentence):
    '''
    단어를 정수 인덱스로 변환
    Vocabulary 구축, word to index mapping
    Args :
        sentence : 문장 리스트
    returns :
        Tokenizer : keras Tokenizer 객체
    '''
    # 토크나이저로 단어사전 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='', # 이미 전처리가 되었으므로 필터 비활성화
        oov_token='<UNK>'
    )
    # 각 문장(한국어 문장, 영어 문장)으로 단어사전 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer

In [ ]:
# 한국어 영어 각각의 토크나이저 생성
korean_tokenizer = create_tokenizer(korean_processed)
# 토크나이저를 함수안이 아닌 글로벌 범위에서 생성한경우
english_tokenizer = create_tokenizer(english_processed)

# korean_tokenizer.word_index

# 단어사전크기 확인
# 단어사전 크기 확인하는 이유 ^
korean_vocab_size = len(korean_tokenizer.word_index) + 1
# UNK 토큰을 고려하여 +1 ^
# korean_tokenizer.word_index를 직접 조회해보니
# 이미 추가되어 있는 것같던데.. 추가하는 이유 아직 잘모르겠다.
english_vocab_size = len(english_tokenizer.word_index) + 1
print(f'한국어 단어사전 크기 : {korean_vocab_size}')
print(f'영어 단어사전 크기 : {english_vocab_size}')

한국어 단어사전 크기 : 25
영어 단어사전 크기 : 30


In [ ]:
# 1.3 점수시퀀스 변경
def padding_sentences(tokenizer, sentences, maxlen):
    '''
    문장을 고정길이의 정수 시퀀스로 변환
    padding : 짧은 문장을 동일길이로 맞추는
    Args :
        tokenizer
        sentence : 전처리된 문장 리스트
        maxlen : 가장 긴 문장의 길이
    Retuens :
        패딩된 점수 시퀀스 배열 tf의 pad_sequnce
    '''
    # texts_to_sequences
    sequnece = tokenizer.texts_to_sequences(sentences) # [i love you] -> [1,5,7]
    # 길이 맞추기 (패딩 추가)
    # pad_sequences
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequnece,
        maxlen = maxlen,
        padding = 'post'
    )
    return padded

#### 패딩 기법 pre / post 어느것을 선택 할지의 대한 기준
- 딥러닝 RNN / LSTM / GUR 전통적인 시계열 처리 기법
    - pre
    - RNN 앞에서부터 읽어 -> 실제 단어 뒤쪽에 몰려있다 -> hidden state 유효한 문맥을 포함 -> h_last에 정보가 압축
- Transformer 계열은 post
    - 위치정보 앞 -> 뒤 순서를 그대로 유지해야 자연스럽다 -> 뒤쪽 패딩은 .. masking 처리 ^


In [ ]:
# 최대 시퀀스의 길이 조회 및 변수에 해당 값 저장 (가장 긴 문장기준)
max_korean_len = max(len(s.split()) for s in korean_processed)
max_english_len = max(len(s.split()) for s in english_processed)
print('시퀀스의 최대길이')
print(f'한국어 : {max_korean_len}')
print(f'영어 : {max_english_len}')


# 만들어 놓은 함수로 패딩 수행 : padding_sentence
korean_tensor = padding_sentences(korean_tokenizer,korean_processed,max_korean_len)
english_tensor = padding_sentences(english_tokenizer,english_processed,max_english_len)

print(f'인코딩 첫번째 문장')
print(f'한국어 패딩 문장 : {korean_processed[2]}')
print(f'영어 패딩 문장{english_processed[2]}')
print(f'한국어 패딩 문장 : {korean_tensor[2]}')
print(f'영어 패딩 문장{english_tensor[2]}')

###
# 1.4
BUFFER_SIZE = len(korean_tensor)
BATCH_SIZE = 2

# tensorflow Dataset 객체
# 배치 처리와 셔플
# 만든 데이터 2개를 묶어 모델에 한번에 넣기위한 데이터 셋 세팅
# 묶을때 batch사용
# from_tensor_slice ^
dataset = tf.data.Dataset.from_tensor_slices((korean_tensor, english_tensor))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True) #^
print('데이터 셋 준비완료')
print(f'전체샘플수 : {len(korean_tensor)}')
print(f'배치크기 : {BATCH_SIZE}')
print(f'배치 수 :{len(korean_tensor) // BATCH_SIZE}')
    # 배치 수 = 입력 데이테 갯수 // 배치사이즈

시퀀스의 최대길이
한국어 : 3
영어 : 5
인코딩 첫번째 문장
한국어 패딩 문장 : <start>저는   학생입니다<end>
영어 패딩 문장<start>i   am   a   student<end>
한국어 패딩 문장 : [2 7 0]
영어 패딩 문장[ 4 11  5 12  0]
데이터 셋 준비완료
전체샘플수 : 10
배치크기 : 2
배치 수 :5


In [10]:
# from_tensor_slice((korean_tensor, english_tensor)): 번역할 문장, 번역된 문장끼리 묶기
#   - 한국어/영어 텐서를 '샘플 단위'로 잘라서 Dataset으로 만듦
#   - (입력 1개, 정답 1개) 형태의 pair를 샘플 하나로 구성
#   - 즉, 각 문장 쌍이 하나의 학습 샘플이 됨

# shuffle(BUFFER_SIZE): 
#   - 전체 샘플 순서를 무작위로 섞어서
#   - 학습이 특정 순서에 치우치지 않도록 함
#   - 셔플을 얼마나 강하게 섞을 것인가의 파리미터에
#     텐서길이만큼의 숫자를 입력한다는 것은 완전 랜덤하게 섞겠다는 말

# batch(BATCH_SIZE, drop_remainder=True): 모델에 넣기위한 데이터 묶음 만들기
#   - 2개씩들어가게 해서 아까 묶어놓은 문장끼리 짝지어 잘 들어가도록 ^ 왜 두번그렇게 하지?
#   - 여러 샘플을 한 번에 묶어서 모델에 넣기 위한 batch 단위를 생성
#   - BATCH_SIZE=2 → 샘플 2개씩 묶어서 모델이 동시에 처리하도록 함
#   - drop_remainder=True → 마지막 배치가 2개보다 적으면 버려서
#                          모든 batch가 동일한 크기(2)를 유지하도록 함
#   - 안정적 학습과 텐서 shape 일관성을 위함

# 한 줄 요약:
#   from_tensor_slice는 "샘플 하나 만들기"
#   batch는 "샘플 여러 개 묶어서 모델에 넣기"

```
-- step2 --
seq2seq 구조
'''
Encoder
    입력문장 -> Embedding -> LSTM + Hidden States
Decoder
    <start> 토큰 -> Embedding -> LSTM + context -> 단어예측
    예측단어 -> 단어입력 -> 반복 (<end> 나올때 까지)
```

In [ ]:
# 2-1
# Encoder 클래스
class Encoder(tf.keras.Model): # Model 상속
    '''
    입력문장을 고차원 벡터로 압축
    Embedding -> LSTM -> Hidden States 출력
    구조 :
        입력(정수시퀀스) -> Embedding -> LSTM 모든 타임스텝의 출력
    '''
    def __init__(self,vocab_size,embedding_dim,enc_units,batch_size):
        '''
        Args :
            vocab_size : 단어사전크기(임베딩 테이블 크기)
            embedding_dim : 임베딩 벡터 차원
            enc_units : LMS 유닛수(hidden state 차원)
            batch_seize : 배치크기
        '''
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        # Embedding Layer : 정수 -> 밀집 벡터(Dense Vector)
        # 학습 과정을 통해 단어 간 의미적 관계학습
        # -> 유사 단어는 가까운 벡터공간에 위치
        # 단점 : 훈련필요(사전학습 또는 임베딩을 학습)
        
        # 희소벡터 :
        # 대부분의 값이 0(대표적인 원-핫 벡터)
        # 차원이 크다
        # 단어간 충돌이 없다
        # 단어간 유사도 표현 못함
        
        # 밀집 벡터 :
        # 학습기반(Word2Vec Glovw Embedding)
        
        # 희소벡터 :
        # 규칙기반(원핫 Bow) - 초창기 자연어 모델
        
        # mask_zero = True ^
        # 패딩에 마스크 처리를 해서 모델이 해석하지 않게 한다. 
        # ^ 
        
        # embedding 레이어 생성
        self.embedding = tf.keras.layers.Embedding(
            vocab_size,embedding_dim, mask_zero = True #^
        )
        
        # LSTM  레이어 생성
        self.lstm = tf.keras.layers.LSTM(
            enc_units,
            return_sequences = True, # [batch, seq_len_units] ^
            return_state = True, # (output, h, c) ^
            recurrent_initializer = 'glorot_uniform' # 가중치 초기화 설정
        )
        
    def call(self, x, hidden):
        '''
        입력 시퀀스를 처리해서 hidden states 생성
        Args :
            x : 입력 시퀀스[ batch_size, seq_len]
            hidden : 초기 hidden state(첫 호출시 0벡터)
        Returns :
        '''
        # 위에서 만든 레이어를 가져와 인코딩
        # 1단계 : Embedding
        # [batch, seq_len]-> [batch,seq_len, embedding_dim]
        x = self.embedding(x)
        
        # 2단계 : LSTM 처리
        # output : 모든타임스텝의 출력(Attenton에서 사용)
        # state_h : 마지막 hidden state(디코더 초기화)
        # state_c : 마지막 cell state(LSTM 내부 메모리)
        output, state_h, state_c = self.lstm(x,initial_state=hidden)
        return output,state_h,state_c
    
    def initialize_hidden_state(self):
        '''
        인코더의 초기 hidden state 0 으로 초기화
        Returns :
            두개의 0텐서 [batch, enc_unit] ) (h와 c)
        '''
        return [
            tf.zeros((self.batch_size, self.enc_units)),
            tf.zeros((self.batch_size, self.enc_units))
        ]

In [13]:
# 위 코드 궁금한점 정리 :
# LSTM에서 Hidden State가 중요한 이유?
# 파이썬 콜백함수에 대하여?

In [27]:
# 2-2 Encoder 테스트
embedding_dim = 2
units = 512 # LSTM unit 수
# encoder 객체
encoder = Encoder(korean_vocab_size,embedding_dim,units,BATCH_SIZE)

# 샘플 데이터
sample_input_batch, sample_target_batch = next(iter(dataset))
initial_hidden = encoder.initialize_hidden_state()

# encorder 클래스에 forward-> call 호출해서 결과를 얻어야....
# why call호출을 하는지? 우리는 모델을 funtional API 방식
# 객체를 함수처럼 사용하면 된다.
sample_output, sample_h, sample_c = encoder(sample_input_batch,initial_hidden)

print(f'{sample_input_batch.shape}') # [batch, korean_seq_len]
print(f'{sample_target_batch.shape}') # [batch, english_seq_len]
print(f'{sample_h.shape}') # [batch,units]
print(f'{sample_c.shape}') # [batch,units]

(2, 3)
(2, 5)
(2, 512)
(2, 512)
